# Homework 8

> I pledge my Honor that I have abided by the Stevens Honor System. - Joshua Schmidt

imports

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "hw8")

In [6]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [3]:
# load data
data = sc.textFile('re_u.data')
data.count()

100000

In [4]:
data.take(5)

['196,242,3', '186,302,3', '22,377,1', '244,51,2', '166,346,1']

In [7]:
ratings = data.map(lambda line: line.split(',')).map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])))
ratings.count()

100000

In [8]:
ratings.take(5)

[Rating(user=196, product=242, rating=3.0),
 Rating(user=186, product=302, rating=3.0),
 Rating(user=22, product=377, rating=1.0),
 Rating(user=244, product=51, rating=2.0),
 Rating(user=166, product=346, rating=1.0)]

In [9]:
# alternating least squares
rank = 10
num_iter = 10
model = ALS.train(ratings, rank, num_iter)

In [10]:
test_data = ratings.map(lambda elem: (elem[0], elem[1]))
predictions = model.predictAll(test_data).map(lambda pred: ((pred[0], pred[1]), pred[2]))
ratings_predictions = ratings.map(lambda rating: ((rating[0], rating[1]), rating[2])).join(predictions)
mse = ratings_predictions.map(lambda elem: (elem[1][0] - elem[1][1]) ** 2).mean()
print(f'mean squared error: {mse:.3f}')

mean squared error: 0.483
